In [1]:
%cd ..

/nfs/scistore16/edelgrp/fzimin/depth-poset


/mnt/nfs/clustersw/Debian/bookworm/jupyterhub/1.0/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import itertools
import numpy as np
import pandas as pd

# Generate Parameters

In [3]:
ns = np.concatenate([np.arange(4, 12, 1), np.arange(12, 165, 8)])
ds = [1, 2, 3, 4]
repeat = 10

In [4]:
params = '\n'.join(['\n'.join([f'{d} {n}' for _ in range(repeat)]) for n, d in itertools.product(ns, ds)])

In [5]:
with open('params.txt', 'w') as file:
    file.write(params)

In [6]:
def get_param_table(s):
    # 
    param_table = []
    for i, s in enumerate(params.split('\n')):
        d, n = s.split()
        param_table.append({'i': i, 'd': int(d), 'n': int(n)})
    param_table = pd.DataFrame(param_table)
    return param_table

In [7]:
def find_consecutive_ranges(arr):
    arr = np.asarray(arr)
    diffs = np.diff(arr)  # Compute differences between consecutive elements
    breaks = np.where(diffs != 1)[0] + 1  # Find where the difference is not 1
    indices = np.concatenate(([0], breaks, [len(arr)]))  # Include start and end of array
    ranges = [(indices[i], indices[i + 1]) for i in range(len(indices) - 1) if indices[i + 1] - indices[i] > 1]
    return ranges

In [8]:
def get_slurm_comand(s, condition=lambda row: True, script_path='slurm/calculate_scores_on_random_alpha_complexes.sh'):
    # 
    param_table = get_param_table(s)
    param_table = param_table[param_table.apply(condition, axis=1)]
    ss = []
    for i0, i1 in find_consecutive_ranges(param_table['i'].values):
        j0, j1 = param_table['i'].values[[i0, i1 - 1]]
        ss.append(f'sbatch --array={j0}-{j1} {script_path}')
    return '\n'.join(ss)

In [9]:
print(get_slurm_comand(params))

sbatch --array=0-1119 slurm/calculate_scores_on_random_alpha_complexes.sh


In [10]:
condition = lambda row: (row['n'] > 36) and  (row['n'] < 60)
print(get_slurm_comand(params, condition)) # This should not start with 0

sbatch --array=480-559 slurm/calculate_scores_on_random_alpha_complexes.sh


# Clear Results

In [ ]:
assert False

In [ ]:
import os
import pickle as pkl

from tqdm import tqdm

In [11]:
condition = lambda result: (result['n'] <= 12) or (result['n'] == 44) or (result['n'] == 52)
print(get_slurm_comand(params, condition))

sbatch --array=0-359 slurm/calculate_scores_on_random_alpha_complexes.sh
sbatch --array=480-559 slurm/calculate_scores_on_random_alpha_complexes.sh


In [ ]:
path = 'results/scores-on-random-alpha-complexes/'

filenames = np.char.add(path, os.listdir(path))
corresponds_the_condition = np.zeros(len(filenames), dtype=bool)
corrupted_files = np.zeros(len(filenames), dtype=bool)

for i, filename in tqdm(enumerate(filenames), total=len(filenames)):
    try:
        with open(filename, 'rb') as file:
            result = pkl.load(file)
        corresponds_the_condition[i] = condition(result)
    except EOFError:
        corrupted_files[i] = True

In [ ]:
files_to_remove = filenames[corresponds_the_condition|corrupted_files]
print(f'There should be {len(files_to_remove)} removed.')
print(f'There are {corresponds_the_condition.sum()} files, corresponding the condition and {corrupted_files.sum()} files corrupted.')

In [ ]:
def remove_files(file_list):
    for filename in tqdm(file_list, total=len(file_list)):
        try:
            os.remove(filename)
        except FileNotFoundError:
            print(f"File not found: {filename}")
        except PermissionError:
            print(f"Permission denied: {filename}")
        except Exception as e:
            print(f"Error removing {filename}: {e}")

In [ ]:
remove_files(files_to_remove)

In [ ]:
print(get_slurm_comand(s, condition))